# Reshaping Arrays (Advanced)

This notebook gives **advanced (but not too much)** practice problems **with solutions** on NumPy reshaping.

You will practice:
- `reshape` with `-1`, and safe shape inference
- Views vs copies (`reshape`, `ravel`, `flatten`, `copy`)
- Order (`order='C'` vs `order='F'`)
- Reshaping after slicing (contiguity)
- Axis manipulation (`transpose`, `moveaxis`, `swapaxes`, `expand_dims`, `squeeze`)
- Combining/splitting arrays (`stack`, `concatenate`, `split`)

Best practices used here:
- Use `assert` checks to validate shapes and content
- Prefer readable variable names + comments
- Avoid silent assumptions about memory layout


In [1]:
import numpy as np

np.set_printoptions(linewidth=120)

## Problem 1 — Reshape with `-1` (and verify)

Create a 1D array `x` containing integers `0..59`.

1) Reshape it into `A` with shape `(5, -1)`.
2) Then reshape `A` into `B` with shape `(-1, 3)`.
3) Verify:
- `A.shape == (5, 12)`
- `B.shape == (20, 3)`
- `B[0] == [0, 1, 2]`
- `B[-1] == [57, 58, 59]`

### Solution

In [2]:
x = np.arange(60)

A = x.reshape(5, -1)
B = A.reshape(-1, 3)

assert A.shape == (5, 12)
assert B.shape == (20, 3)
assert np.array_equal(B[0], np.array([0, 1, 2]))
assert np.array_equal(B[-1], np.array([57, 58, 59]))

A, B[:2], B[-2:]

(array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
        [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47],
        [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]]),
 array([[0, 1, 2],
        [3, 4, 5]]),
 array([[54, 55, 56],
        [57, 58, 59]]))

## Problem 2 — View vs Copy (`reshape`, `ravel`, `flatten`)

Given `M = arange(12).reshape(3, 4)`, create:
- `v1` as a 1D view of `M` (when possible)
- `c1` as a guaranteed independent 1D copy

Then mutate `v1[0] = 999` and verify it changes `M[0, 0]`.
Then mutate `c1[1] = 555` and verify it **does not** change `M`.

Hint: compare `np.ravel` and `np.ndarray.flatten`.

### Solution

In [3]:
M = np.arange(12).reshape(3, 4)

v1 = np.ravel(M)        # returns a view when possible
c1 = M.flatten()        # always returns a copy

v1[0] = 999
assert M[0, 0] == 999

old = M.copy()
c1[1] = 555
assert np.array_equal(M, old)  # unchanged

M, v1[:6], c1[:6]

(array([[999,   1,   2,   3],
        [  4,   5,   6,   7],
        [  8,   9,  10,  11]]),
 array([999,   1,   2,   3,   4,   5]),
 array([  0, 555,   2,   3,   4,   5]))

## Problem 3 — Reshape order: `C` vs `F`

Create `x = arange(12)` and reshape it into `(3, 4)` in two ways:
- `C = x.reshape(3, 4, order='C')`
- `F = x.reshape(3, 4, order='F')`

1) Print both arrays.
2) Verify that `C[0, :] == [0, 1, 2, 3]`.
3) Verify that `F[:, 0] == [0, 1, 2]`.

### Solution

In [4]:
x = np.arange(12)

C = x.reshape(3, 4, order='C')
F = x.reshape(3, 4, order='F')

assert np.array_equal(C[0, :], np.array([0, 1, 2, 3]))
assert np.array_equal(F[:, 0], np.array([0, 1, 2]))

C, F

(array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]),
 array([[ 0,  3,  6,  9],
        [ 1,  4,  7, 10],
        [ 2,  5,  8, 11]]))

## Problem 4 — Reshaping after slicing (contiguity)

Slicing can create **non-contiguous** views. Some reshapes will fail unless you copy.

1) Create `A = arange(24).reshape(4, 6)`.
2) Take every other column: `S = A[:, ::2]` (shape `(4, 3)`).
3) Try to reshape `S` into shape `(2, 6)`.
   - If it fails, fix it using **one** of the following:
     - `S.copy()`
     - `np.ascontiguousarray(S)`

Finally, verify the reshaped result has the same elements as `S.ravel(order='C')`.

### Solution

In [5]:
A = np.arange(24).reshape(4, 6)
S = A[:, ::2]

assert S.shape == (4, 3)

try:
    R = S.reshape(2, 6)
except ValueError:
    # Best practice: make contiguity explicit when needed.
    R = np.ascontiguousarray(S).reshape(2, 6)

assert R.shape == (2, 6)
assert np.array_equal(R.ravel(order='C'), S.ravel(order='C'))

A, S, R

(array([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]]),
 array([[ 0,  2,  4],
        [ 6,  8, 10],
        [12, 14, 16],
        [18, 20, 22]]),
 array([[ 0,  2,  4,  6,  8, 10],
        [12, 14, 16, 18, 20, 22]]))

## Problem 5 — Axis moves: from (N, H, W, C) to (N, C, H, W)

You have image-like data `X` in NHWC format `(N, H, W, C)`.

1) Create `X = arange(2*3*4*5).reshape(2, 3, 4, 5)`.
2) Convert to `Y` in NCHW format `(N, C, H, W)`.
3) Verify:
- `Y.shape == (2, 5, 3, 4)`
- The element `X[1, 2, 3, 4]` ends up at `Y[1, 4, 2, 3]`.

Use either `transpose` or `moveaxis`.

### Solution

In [6]:
X = np.arange(2 * 3 * 4 * 5).reshape(2, 3, 4, 5)

# NHWC -> NCHW
Y = np.transpose(X, (0, 3, 1, 2))

assert Y.shape == (2, 5, 3, 4)
assert X[1, 2, 3, 4] == Y[1, 4, 2, 3]

X.shape, Y.shape, (X[1, 2, 3, 4], Y[1, 4, 2, 3])

((2, 3, 4, 5), (2, 5, 3, 4), (np.int64(119), np.int64(119)))

## Problem 6 — Add and remove singleton dimensions (`expand_dims`, `squeeze`)

Given `v = arange(10)` (shape `(10,)`):

1) Make it a column vector of shape `(10, 1)` named `col`.
2) Then make it shape `(1, 10, 1)` named `weird`.
3) Finally, remove singleton axes to get back to shape `(10,)` named `back`.

Verify shapes at each step.

### Solution

In [7]:
v = np.arange(10)

col = v[:, np.newaxis]
weird = np.expand_dims(col, axis=0)  # adds axis at the front
back = np.squeeze(weird)

assert v.shape == (10,)
assert col.shape == (10, 1)
assert weird.shape == (1, 10, 1)
assert back.shape == (10,)
assert np.array_equal(back, v)

v.shape, col.shape, weird.shape, back.shape

((10,), (10, 1), (1, 10, 1), (10,))

## Problem 7 — Combine, then reshape, then split back

You have 3 sensors each producing a `(4, 2)` matrix. You want a single tensor `(3, 4, 2)`.

1) Create three arrays `s0, s1, s2` where:
- `s0 = arange(8).reshape(4, 2)`
- `s1 = arange(100, 108).reshape(4, 2)`
- `s2 = arange(200, 208).reshape(4, 2)`
2) Stack them into `T` with shape `(3, 4, 2)`.
3) Flatten `T` to `flat` (shape `(24,)`) using a method that returns a **view when possible**.
4) Split `T` back into the original 3 matrices and verify exact equality.

### Solution

In [8]:
s0 = np.arange(8).reshape(4, 2)
s1 = np.arange(100, 108).reshape(4, 2)
s2 = np.arange(200, 208).reshape(4, 2)

T = np.stack([s0, s1, s2], axis=0)
assert T.shape == (3, 4, 2)

flat = np.ravel(T)
assert flat.shape == (24,)

u0, u1, u2 = np.split(T, indices_or_sections=3, axis=0)
u0 = np.squeeze(u0, axis=0)
u1 = np.squeeze(u1, axis=0)
u2 = np.squeeze(u2, axis=0)

assert np.array_equal(u0, s0)
assert np.array_equal(u1, s1)
assert np.array_equal(u2, s2)

T.shape, flat[:8], u2

((3, 4, 2),
 array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([[200, 201],
        [202, 203],
        [204, 205],
        [206, 207]]))

## Problem 8 — Reshape + `swapaxes` to block-structure a matrix

You are given a 2D matrix `A` of shape `(6, 8)`. Interpret it as a grid of blocks:
- 3 block-rows, each of height 2
- 4 block-columns, each of width 2

Goal: Produce a 4D array `B` of shape `(3, 4, 2, 2)` such that `B[i, j]` is the `(2, 2)` block.

Steps (suggested):
1) Reshape `(6, 8)` into `(3, 2, 4, 2)`
2) Swap axes to get `(3, 4, 2, 2)`

Verify that the top-left block equals `A[0:2, 0:2]` and the last block equals `A[4:6, 6:8]`.

### Solution

In [9]:
A = np.arange(6 * 8).reshape(6, 8)

tmp = A.reshape(3, 2, 4, 2)
B = tmp.swapaxes(1, 2)

assert B.shape == (3, 4, 2, 2)
assert np.array_equal(B[0, 0], A[0:2, 0:2])
assert np.array_equal(B[2, 3], A[4:6, 6:8])

A, B[0, 0], B[2, 3]

(array([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47]]),
 array([[0, 1],
        [8, 9]]),
 array([[38, 39],
        [46, 47]]))

## Quick reference (cheat sheet)

- `reshape(...)` returns a **view when possible**, otherwise may copy or fail if the view isn't compatible
- `ravel()` returns a **view when possible**, otherwise copies
- `flatten()` **always copies**
- Slices are often **views** (and may be non-contiguous)
- If reshape fails after slicing, use `np.ascontiguousarray(x)` or `x.copy()`
- `transpose/moveaxis/swapaxes` reorders axes (often view-like)
- `expand_dims` / `np.newaxis` adds size-1 axes; `squeeze` removes them
